<a href="https://colab.research.google.com/github/rsrini7/Colabs/blob/main/PydanticAI_GoogleSheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Create Your Own AI Agent To Automate Google Sheets | PydanticAI Tutorial](https://www.youtube.com/watch?v=tURxphunUyk)

Important Notes for Colab:

credentials.json: You will need to upload your credentials.json file to your Colab environment. The code assumes it's in the root directory of your Colab session.

Spreadsheet ID: You'll need to provide the ID of the Google Sheet you want the agent to work with.

LLM API Keys (If not using Gemini with OAuth): The primary example uses Google Gemini with OAuth, which is handled by credentials.json. If you switch to other models like OpenAI GPT-4o or Anthropic Claude, you would typically need to set their API keys.

For Colab, you'd use the "Secrets" manager (key icon on the left) and access them via userdata.get('YOUR_API_KEY_NAME'). However, since the video focuses on Gemini via Google Cloud setup, we'll assume that authentication path.

In [2]:
!pip install pydantic_ai google-api-python-client google-auth-httplib2 google-auth-oauthlib nest_asyncio -q

Upload credentials.json and Set Spreadsheet ID
Run this cell. It will prompt you to upload your credentials.json file.

Then, replace "YOUR_SPREADSHEET_ID_HERE" with the actual ID of your Google Sheet.

The Spreadsheet ID can be found in the URL of your Google Sheet: https://docs.google.com/spreadsheets/d/SPREADSHEET_ID/edit

In [3]:
# Replace with your Google Sheet ID
# SPREADSHEET_ID = "YOUR_SPREADSHEET_ID_HERE"
# --- USER CONFIGURATION ---
# Example: SPREADSHEET_ID = "1dkIeTJIezJ2mBBS7U3TUxCdAeDXJ4cO3HXfJg_ozOzPo"
SPREADSHEET_ID = "1SJyM0PRFyb84NRTcOu5OXTFlm_SlGCxKAcID4ciaRjg"

In [4]:
from google.colab import files
import os

# Upload credentials.json
if not os.path.exists('credentials.json'):
    uploaded = files.upload()
    for fn in uploaded.keys():
        print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')
        # Ensure it's named credentials.json if uploaded with a different name
        if fn != 'credentials.json':
            os.rename(fn, 'credentials.json')
            print(f'Renamed "{fn}" to "credentials.json"')
else:
    print('credentials.json already exists.')

if SPREADSHEET_ID == "YOUR_SPREADSHEET_ID_HERE":
    print("ERROR: Please update SPREADSHEET_ID with your actual Google Sheet ID.")
else:
    print(f"Using SPREADSHEET_ID: {SPREADSHEET_ID}")

# Create a directory for token files if it doesn't exist
TOKEN_DIR = 'token_files'
if not os.path.exists(TOKEN_DIR):
    os.makedirs(TOKEN_DIR)
    print(f"Created directory: {TOKEN_DIR}")

credentials.json already exists.
Using SPREADSHEET_ID: 1SJyM0PRFyb84NRTcOu5OXTFlm_SlGCxKAcID4ciaRjg
Created directory: token_files


In [10]:
# load_models.py
# This script defines the LLM models you might use. The tutorial focuses on Gemini.
from google.colab import userdata
from pydantic_ai.models.gemini import  GeminiModel
# from pydantic_ai.models import OpenAIModel, OllamaModel, GroqModel, AnthropicModel
# from google.colab import userdata # Uncomment if you use other models requiring API keys

# --- LLM Model Configuration ---
# The tutorial uses Gemini, which should be authenticated via the OAuth flow (credentials.json)
# when interacting with Google Cloud services.
# If you use Google AI Studio's Gemini API, you'd need an API key.
# GEMINI_API_KEY = userdata.get('GEMINI_API_KEY') # Example for Google AI Studio Gemini
# GEMINI_MODEL = GeminiModel("gemini-1.5-flash-latest", api_key=GEMINI_API_KEY)

GEMINI_API_KEY_FROM_USERDATA = userdata.get('GEMINI_API_KEY')

if GEMINI_API_KEY_FROM_USERDATA:
  print("Found GEMINI_API_KEY in Colab secrets. Using it for Google AI Studio Gemini.")
  os.environ.setdefault("GEMINI_API_KEY", GEMINI_API_KEY_FROM_USERDATA)
else:
  raise Exception("GEMINI_API_KEY not found in Colab secrets.")

# For Vertex AI Gemini (as likely used in the video with Google Cloud setup):
GEMINI_MODEL = GeminiModel("gemini-1.5-flash-latest") # PydanticAI handles Vertex AI auth if gcloud is setup or credentials are available
                                               # For Colab, the OAuth flow from google_apis.py should cover this.
                                               # Video uses 'gemini-2.0-flash-exp', but 'gemini-1.5-flash-latest' is a common general model.
                                               # You might need to adjust the model name based on availability and your GCP project.

# --- Optional: Other Models (uncomment and configure if needed) ---
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# OPENAI_MODEL = OpenAIModel("gpt-4o-mini", api_key=OPENAI_API_KEY)

# OLLAMA_MODEL = OllamaModel("llama3:8b") # Assumes Ollama server is running and accessible

# GROQ_API_KEY = userdata.get('GROQ_API_KEY')
# GROQ_MODEL = GroqModel("llama3-groq-8b-8192-tool-use-preview", api_key=GROQ_API_KEY)

# ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
# ANTHROPIC_MODEL = AnthropicModel("claude-3-5-sonnet-20240620", api_key=ANTHROPIC_API_KEY)


# For this tutorial, we'll stick to the GEMINI_MODEL as shown in the video's setup context.
# If you encounter issues with the Gemini model string, try "gemini-pro" or other available Gemini models.
# The video showed "gemini-2.0-flash-exp", which might be an older or specific model identifier.
# We are using "gemini-1.5-flash-latest" as a robust default.
# If you have specific access to "gemini-2.0-flash-exp", you can use that.
try:
    # Attempt to use the model string from the video if available
    GEMINI_MODEL_FROM_VIDEO = GeminiModel('gemini-2.0-flash-exp')
    GEMINI_MODEL = GEMINI_MODEL_FROM_VIDEO
    print("Using Gemini model: gemini-2.0-flash-exp (from video)")
except Exception as e:
    print(f"Could not initialize 'gemini-2.0-flash-exp'. Error: {e}")
    print("Falling back to 'gemini-1.5-flash-latest'. Ensure your GCP project has this model enabled or use an API key method.")
    # Fallback if the specific video model isn't accessible directly
    # If using Google AI Studio, you'd provide api_key=userdata.get('GOOGLE_AI_STUDIO_API_KEY')
    GEMINI_MODEL = GeminiModel("gemini-1.5-flash-latest")


print(f"Selected LLM for the agent: {GEMINI_MODEL.model_name}")

Found GEMINI_API_KEY in Colab secrets. Using it for Google AI Studio Gemini.
Using Gemini model: gemini-2.0-flash-exp (from video)
Selected LLM for the agent: gemini-2.0-flash-exp


In [29]:
# google_apis.py
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import warnings

import nest_asyncio
nest_asyncio.apply()

# Suppress the specific UserWarning from google.auth.
warnings.filterwarnings("ignore", message="Your application has authenticated using end user credentials")

def create_service(client_secret_file, api_name, api_version, *scopes, prefix=''):
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]] # Ensure scopes is a list

    creds = None
    working_dir = os.getcwd()
    token_dir = os.path.join(working_dir, 'token_files')
    if not os.path.exists(token_dir):
        os.makedirs(token_dir)

    token_file = os.path.join(token_dir, f'token_{API_SERVICE_NAME}_{API_VERSION}{prefix}.json')

    if os.path.exists(token_file):
        try:
            creds = Credentials.from_authorized_user_file(token_file, SCOPES)
        except Exception as e:
            print(f"Error loading token from file '{token_file}': {e}. Will attempt re-authentication.")
            creds = None

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                print("Refreshing expired token...")
                creds.refresh(Request())
                print("Token refreshed successfully.")
            except Exception as e:
                print(f"Error refreshing token: {e}. Will attempt full re-authentication.")
                creds = None

        if not creds or not creds.valid: # Check again after potential refresh
            if not os.path.exists(CLIENT_SECRET_FILE):
                print(f"ERROR: Client secret file '{CLIENT_SECRET_FILE}' not found. Please upload it in Cell 2.")
                return None
            try:
                # Explicitly set the redirect_uri to 'urn:ietf:wg:oauth:2.0:oob'
                # This is the "Out Of Band" flow, where Google shows you the code.
                flow = InstalledAppFlow.from_client_secrets_file(
                    CLIENT_SECRET_FILE,
                    SCOPES,
                    redirect_uri='urn:ietf:wg:oauth:2.0:oob' # Key change here
                )

                auth_url, _ = flow.authorization_url(prompt='consent', access_type='offline')

                print(f'\n--- MANUAL AUTHENTICATION (OOB FLOW) ---')
                print(f'1. Please visit this URL to authorize this application:\n{auth_url}')
                print("\n2. After authorizing, Google will display an authorization code on the page.")
                print("   Copy that code.")

                code = input('\n3. Enter the authorization code from Google here: ')
                code = code.strip()

                flow.fetch_token(code=code)
                creds = flow.credentials
                print("\nAuthentication successful, token fetched.")

            except Exception as e:
                print(f"Error during manual OAuth (OOB) code exchange: {e}")
                return None

        if creds:
            try:
                with open(token_file, 'w') as token:
                    token.write(creds.to_json())
                print(f"Token saved to {token_file}")
            except Exception as e:
                print(f"Error saving token to '{token_file}': {e}")
        else:
            print("Credentials object is None after authentication attempt. Cannot save token.")
            return None

    if creds and creds.valid:
        try:
            service = build(API_SERVICE_NAME, API_VERSION, credentials=creds, static_discovery=False)
            print(f"{API_SERVICE_NAME} {API_VERSION} service created successfully")
            return service
        except Exception as e:
            print(f'Failed to create service instance for {API_SERVICE_NAME} using obtained credentials.')
            print(e)
            return None
    else:
        print("Failed to obtain valid credentials after the OAuth flow.")
        return None

In [37]:
# sheets_agent.py
import ssl
from typing import Any, Optional, List, Dict
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
from pydantic_ai.models import ModelSettings
from pydantic_ai.exceptions import ModelRetry, UnexpectedModelBehavior
from pydantic_ai.agent import AgentRunResult


# --- Data Structures ---
@dataclass
class SheetsDependencies:
    sheets_service: Any
    spreadsheet_id: str

class SheetsResult(BaseModel):
    request_status: Optional[bool] = Field(default=None, description='Status of the request')
    result_details: str = Field(description='Details of the request result')

# --- Google Sheets Client Initialization ---
def init_google_sheets_client() -> Any:
    CLIENT_SECRET_FILE = 'credentials.json'
    API_NAME = 'sheets'
    API_VERSION = 'v4'
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    service = create_service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)
    return service

# --- Pydantic AI Agent Definition ---
if GEMINI_MODEL:
    sheets_agent = Agent(
        model=GEMINI_MODEL,
        deps_type=SheetsDependencies,
        result_types=SheetsResult,
        system_prompt="""You are a Google Sheets agent to help me manage my Google Sheets' tasks.
    When making API calls, wait 1 second between each request to avoid rate limits.
    If a user asks to create multiple sheets (e.g., "Jan" to "Dec"),
    make separate API calls for each sheet creation.
    If a user asks to delete multiple sheets, make separate API calls for each sheet deletion.
    """,
        model_settings=ModelSettings(timeout=60, retries=3),
    )
else:
    sheets_agent = None
    print("Sheets agent could not be initialized because GEMINI_MODEL is not available.")


# --- Agent Tools (Functions) ---
if sheets_agent:
    @sheets_agent.tool(retries=2)
    def add_sheet(ctx: RunContext[SheetsDependencies], sheet_name: str) -> SheetsResult:
        """Adds a new sheet to an existing Google Spreadsheet.

        Args:
            ctx: Run context containing sheets service and spreadsheet ID.
            sheet_name: Name for the new sheet.
        Returns:
            Response from the API after adding the sheet.
        """
        try:
            print(f"Calling add_sheet to add sheet '{sheet_name}'")
            request_body = {
                'requests': [{'addSheet': {'properties': {'title': sheet_name}}}]
            }
            response = ctx.deps.sheets_service.spreadsheets().batchUpdate(
                spreadsheetId=ctx.deps.spreadsheet_id,
                body=request_body
            ).execute()
            return SheetsResult(request_status=True, result_details=f"Sheet '{sheet_name}' added successfully. Response: {response}")
        except UnexpectedModelBehavior as e:
            return SheetsResult(request_status=False, result_details=f"An error occurred (Model Behavior): {str(e)}")
        except ssl.SSLError as e:
            raise ModelRetry(f"An SSL error occurred: {str(e)}. Please try again.")
        except Exception as e:
            return SheetsResult(request_status=False, result_details=f"An error occurred while adding sheet: {str(e)}")


    @sheets_agent.tool(retries=2)
    def delete_sheet(ctx: RunContext[SheetsDependencies], sheet_name: str) -> SheetsResult:
        """Deletes a sheet from an existing Google Spreadsheet by sheet name.

        Args:
            ctx: Run context containing sheets service and spreadsheet ID.
            sheet_name: Name of the sheet to delete.
        Returns:
            Response from the API after deletion.
        """
        try:
            print(f"Calling delete_sheet to delete sheet '{sheet_name}'")
            sheet_metadata = ctx.deps.sheets_service.spreadsheets().get(
                spreadsheetId=ctx.deps.spreadsheet_id
            ).execute()
            sheets = sheet_metadata.get('sheets', [])
            sheet_id = None
            for sheet in sheets:
                if sheet.get('properties', {}).get('title') == sheet_name:
                    sheet_id = sheet.get('properties', {}).get('sheetId')
                    break
            if not sheet_id:
                return SheetsResult(request_status=False, result_details=f"Sheet '{sheet_name}' not found.")
            request_body = {'requests': [{'deleteSheet': {'sheetId': sheet_id}}]}
            response = ctx.deps.sheets_service.spreadsheets().batchUpdate(
                spreadsheetId=ctx.deps.spreadsheet_id,
                body=request_body
            ).execute()
            return SheetsResult(request_status=True, result_details=f"Sheet '{sheet_name}' (ID: {sheet_id}) deleted successfully. Response: {response}")
        except UnexpectedModelBehavior as e:
            return SheetsResult(request_status=False, result_details=f"An error occurred (Model Behavior): {str(e)}")
        except ssl.SSLError as e:
            raise ModelRetry(f"An SSL error occurred: {str(e)}. Please try again.")
        except Exception as e:
            return SheetsResult(request_status=False, result_details=f"An error occurred while deleting sheet: {str(e)}")


    @sheets_agent.tool(retries=2)
    def list_sheets(ctx: RunContext[SheetsDependencies]) -> SheetsResult:
        """Lists all sheets in the Google Spreadsheet.

        Args:
            ctx: Run context containing sheets service and spreadsheet ID.
        Returns:
            A list of dictionaries, each containing the 'id' and 'name' of a sheet.
        """
        try:
            print("Calling list_sheets")
            sheet_metadata = ctx.deps.sheets_service.spreadsheets().get(
                spreadsheetId=ctx.deps.spreadsheet_id
            ).execute()
            sheets = sheet_metadata.get('sheets', [])
            if not sheets:
                return SheetsResult(request_status=True, result_details="No sheets found.")
            sheet_list_details = []
            for sheet in sheets:
                properties = sheet.get('properties', {})
                sheet_id = properties.get('sheetId')
                sheet_title = properties.get('title')
                if sheet_id is not None and sheet_title is not None:
                     sheet_list_details.append({'id': sheet_id, 'name': sheet_title})
            return SheetsResult(request_status=True, result_details=f"Sheets found: {sheet_list_details}")
        except UnexpectedModelBehavior as e:
            return SheetsResult(request_status=False, result_details=f"An error occurred (Model Behavior): {str(e)}")
        except ssl.SSLError as e:
            raise ModelRetry(f"An SSL error occurred: {str(e)}. Please try again.")
        except Exception as e:
            return SheetsResult(request_status=False, result_details=f"An error occurred while listing sheets: {str(e)}")


# --- Main Execution Block ---
if __name__ == "__main__":
    if SPREADSHEET_ID == "YOUR_SPREADSHEET_ID_HERE":
        print("Please set the SPREADSHEET_ID in Cell 2 before running this agent.")
    elif not sheets_agent:
        print("Google Sheets Agent could not be initialized. Please check model configuration in Cell 3.")
    else:
        service_instance = init_google_sheets_client()
        if service_instance:
            deps = SheetsDependencies(sheets_service=service_instance, spreadsheet_id=SPREADSHEET_ID)
            print(f"Google Sheets Agent initialized. Interacting with Spreadsheet ID: {SPREADSHEET_ID}")
            print("Type 'exit' to quit.")

            conversation_history = None
            while True:
                prompt_text = input("User: ")
                if prompt_text.lower().strip() == 'exit':
                    print("Exiting agent.")
                    break
                try:
                    agent_run_result: AgentRunResult # Type hint for clarity
                    if conversation_history:
                        agent_run_result = sheets_agent.run_sync(
                            prompt_text.strip(),
                            deps=deps,
                            message_history=conversation_history.all_messages()
                        )
                    else:
                        agent_run_result = sheets_agent.run_sync(prompt_text.strip(), deps=deps)

                    tool_result: Optional[SheetsResult] = None # Initialize to None
                    # Try to get structured tool output from .output
                    if hasattr(agent_run_result, 'output') and isinstance(agent_run_result.output, SheetsResult):
                        tool_result = agent_run_result.output
                    # If .output is not SheetsResult, it might be the direct LLM string response in .response
                    # or the tool result might be directly in .response if no .output attribute exists or is different
                    elif hasattr(agent_run_result, 'response') and isinstance(agent_run_result.response, SheetsResult):
                         tool_result = agent_run_result.response


                    print(f"\nSheets Agent:")
                    if tool_result: # If we successfully got a SheetsResult object
                        if tool_result.request_status is not None:
                            print(f"  Status: {'Success' if tool_result.request_status else 'Failed'}")
                        print(f"  Details: {tool_result.result_details}\n")
                    elif hasattr(agent_run_result, 'response') and isinstance(agent_run_result.response, str):
                        # This handles cases where the LLM responds directly without calling a tool
                        # or if the tool call failed at the LLM decision stage.
                        print(f"  LLM Response: {agent_run_result.response}\n")
                    else:
                        print(f"  Unexpected result format or no tool was called successfully.")
                        print(f"  Raw agent_run_result: {agent_run_result}\n")


                    conversation_history = agent_run_result

                except ModelRetry as e:
                    print(f"ModelRetry Exception: {str(e)}")
                except UnexpectedModelBehavior as e:
                     print(f"UnexpectedModelBehavior Exception: {str(e)}")
                except NameError as e: # Specifically catch NameError for AgentRunResult if import is missed
                    print(f"A NameError occurred: {e}. This might be due to a missing import for 'AgentRunResult'.")
                    print("Please ensure 'from pydantic_ai.ai_models import AgentRunResult' is at the top of this script.")
                    break # Exit the loop if critical import is missing
                except Exception as e:
                    print(f"An unexpected error occurred in the main loop: {e}")
                    import traceback
                    traceback.print_exc()
        else:
            print("Failed to initialize Google Sheets service. Agent cannot start.")

sheets v4 service created successfully
Google Sheets Agent initialized. Interacting with Spreadsheet ID: 1SJyM0PRFyb84NRTcOu5OXTFlm_SlGCxKAcID4ciaRjg
Type 'exit' to quit.
User: create new sheet called "Srini"
Calling add_sheet to add sheet 'Srini'

Sheets Agent:
  Unexpected result format or no tool was called successfully.
  Raw agent_run_result: AgentRunResult(output='OK. I\'ve created a sheet named "Srini".\n')

User: get number of sheets and display
Calling list_sheets

Sheets Agent:
  Unexpected result format or no tool was called successfully.
  Raw agent_run_result: AgentRunResult(output="There are 3 sheets in the spreadsheet. They are named 'Summary', 'Transactions', and 'Srini'.\n")

User: delete sheet named Srini
Calling delete_sheet to delete sheet 'Srini'

Sheets Agent:
  Unexpected result format or no tool was called successfully.
  Raw agent_run_result: AgentRunResult(output='OK. I\'ve deleted the sheet named "Srini".\n')

User: how many sheets now ?
Calling list_sheets

